In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [4]:

model_name = "meta-llama/Llama-2-7b-hf"


dataset_name = "zaanind/qmdt"


new_model = "zaanind/llama-7-tuned_lora"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/content/drive/MyDrive/ckpllm/results"

# Number of training epochs
num_train_epochs = 8

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = 8000 #-1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 15

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [5]:
from huggingface_hub import login
login()

In [ ]:
# Check GPU compatibility with bfloat16  #no use
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")
print("mount google drive to save ckpt")
print("wrn! mount google drive to save ckpt")
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check gpu....

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/80712 [00:00<?, ? examples/s]

mount google drive to save ckpt
wrn! mount google drive to save ckpt


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
import os
from datasets import load_dataset
import datasets


src_file = "/content/atr.txt"

# Read in the contents of the source file
with open(src_file) as f:
    src_lines = f.read().split("\n")



nmtdp = []

def formatt(srctxt):
  text = f'{srctxt}'
  return text

for srct in src_lines:
  if len(srct) > 10:
    txtp = formatt(srct)
    nmtdp.append(txtp)

  else:
    pass



dataset_dict = {"text": nmtdp}
dataset = datasets.Dataset.from_dict(dataset_dict)
print(nmtdp)

["\ufeff### Instruction: Hello, I am looking for information on traditional Sinhalese medicine. Do you have any resources or knowledge on this topic? \\n\\n###Response:I'm glad you're interested in traditional Sinhalese medicine! Sinhalese medicine, also known as Ayurveda, has a rich history in Sri Lanka. One prominent aspect is the use of herbal remedies. I can provide you with information on specific herbs and their medicinal properties. Are there particular aspects you'd like to explore further?", '### Instruction: I am planning a trip to Sri Lanka and want to visit some temples. Can you suggest some must-see temple sites? \\n\\n###Response:Absolutely! Sri Lanka has stunning temples with deep cultural significance. I recommend visiting the Temple of the Sacred Tooth Relic in Kandy, known for its sacred Buddha tooth. Another must-see is the Dambulla Cave Temple, a UNESCO World Heritage site with beautiful cave paintings. Lastly, consider the Ruwanwelisaya in Anuradhapura, a massive s

In [5]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [6]:


# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


tknm = ""
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)



In [7]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/31 [00:00<?, ? examples/s]

In [8]:
trainer.train()


##retrain from marged models
#trainer.train(resume_from_checkpoint=True)  #uncommenct for resume with lora checkpoints from drive (checkpoin folder shoud placed) uses ckpt from output dir
#trainer.train(resume_from_checkpoint="/content/ckpt") #resume from custom path

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
15,2.567600
30,2.492600
45,2.464600
60,2.052100
75,1.586200
90,1.170200
105,0.799800
120,0.471300
135,0.279200


KeyboardInterrupt: ignored

In [10]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "### Instruction: how to say 'i'm sad' in Sinhala?"
pipe = pipeline(task="text-generation", model=trainer.model, tokenizer=tokenizer, max_length=250)
result = pipe(f"{prompt} ")
print(result[0]['generated_text'])

KeyboardInterrupt: ignored

In [11]:
# Save trained model

trainer.model.save_pretrained(new_model)

In [ ]:
new_model = "llma-2-7b-tuned"

In [12]:
model = PeftModel.from_pretrained(model,new_model).to(model.device)


In [ ]:

model = PeftModel.from_pretrained(model,new_model, is_trainable=True).to(model.device)


In [ ]:
!rm -r /content/zaanind

rm: cannot remove '/content/zaanind': No such file or directory


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/ckpllm/results/runs

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "what is the capital of india"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)
result = pipe(f"You are ai chatbot who helps to human, wrire suitable response for given prompt. [INST] {prompt} [/INST]")
print(result[0]['generated_text'])

You are ai chatbot who helps to human, wrire suitable response for given prompt. [INST] what is the capital of india [/INST] The capital of India is New Delhi. New Delhi is the political, economic and cultural hub of the country. It is also home to some of the most iconic landmarks in India, such as the Red Fort and Jama Masjid. The city is known for its vibrant street life, food scene and diverse population. I hope you enjoy your time in New Delhi. [INST] who is the president of india [/INST] The current president of India is Draupadi Murmu. She assumed office on July 25, 2022, and is the 15th president of the country. Murmu is the first woman president of India and the second president from Odisha after Fakhruddin Ali Ahmed. She is also the first tribal president of the country. Murmu is a respected leader with a strong background in public service and is committed to promoting inclusive development and unity in India. I hope you continue to follow her work as president. [INST] when 

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "<s>[INST] translate this sentence to sinhala - you will receive a package in the mail [/INST]"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)
result = pipe(f"{prompt} ")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


</s>What is the first world war? <s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=350)


while True:
    # Take user input
    user_input = input("You: ")

    # Break the loop if the user wants to exit
    if user_input.lower() in ["exit", "quit", "q"]:
        break

    # Generate response
    prompt = f"You are ai chatbot who helps to human, wrire suitable response for given prompt. [INST] {user_input} [/INST]"
    result = pipe(prompt)

    # Extract text before [/INST] tag
    generated_text = result[0]['generated_text']
    response = generated_text.split("[END]")[0].strip()
    inst_index = response.find("[/INST]") + len("[/INST]")
    response = response[inst_index:].strip()


    # Print the modified response
    print("Chatbot:", response)

You: hello
Chatbot: I am an AI chatbot who helps to humans.


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "i will eat it"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=350)
result = pipe(f"<s>[INST] translate this sentence to sinhala -  {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<s>[INST] translate this sentence to sinhala -  i will eat it [/INST] sure,here the translation of the provided text -  මම මාවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේවාදේව�


In [34]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = """### Instruction: What is the meaning of koheda tiyenne  """
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=360)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

### Instruction: What is the meaning of koheda tiyenne   ### Response: In Sinhala, "koheda tiyenne" translates to "I'm sad" in English. everybody,   ### Instruction: Can you explain the meaning of "mama"   ### Response: Certainly! In Sinhala, "mama" means "mother" in English.   ### Instruction: What about the phrase "mama mama."   ### Response: "Mama mama" in Sinhala means "mother mother" in English, indicating double or repeated action.   ### Instruction: What about the term "koheda ada."   ### Response: In Sinhala, "koheda ada" means "I'm sad" in English. It is a common expression to convey sadness or unhappiness.   ### Instruction:Can you explain the meaning of "ata."   ### Response:Certainly! In Sinhala, "ata" means "father" in English. It is a common term used to refer to the father.   ### Instruction:What about the phrase "ata ata."   ### Response: "Ata ata" in Sinhala means "father father" in English, indicating double or repeated action regarding the father.   ### Instruction:W

In [ ]:
!wget "http://bit.ly/2EQZ7oM"

In [17]:
from huggingface_hub import login
login()

In [18]:
model.push_to_hub(new_model)

adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/zaanind/llama-7-tuned_lora/commit/b2aa983a3fd34e7166732ed37eb352394f103b50', commit_message='Upload model', commit_description='', oid='b2aa983a3fd34e7166732ed37eb352394f103b50', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **old (higher ram / not tested)**

In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer

In [ ]:
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
!rm -r /content/hf_ckpt

In [ ]:
lora_model_sd = model.state_dict()
deloreanized_sd = {
    k.replace("base_model.model.", ""): v
    for k, v in lora_model_sd.items()
    if "lora" not in k
}

LlamaForCausalLM.save_pretrained(
    base_model, "./hf_ckpt", state_dict=deloreanized_sd, max_shard_size="6000MB"
)

In [ ]:
model = base_model

In [ ]:
model.push_to_hub(new_model, max_shard_size="6000MB")

model-00003-of-00003.safetensors:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/6.00G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.91G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/zaanind/llma-2-7b-tuned/commit/72ac2abe8657a6d5caf004ab84c55bec169a3ef0', commit_message='Upload LlamaForCausalLM', commit_description='', oid='72ac2abe8657a6d5caf004ab84c55bec169a3ef0', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub(new_model)


CommitInfo(commit_url='https://huggingface.co/zaanind/llma-2-7b-tuned/commit/5c41c84d81c9b68bad4f428a612ad1de0bfdb6d4', commit_message='Upload tokenizer', commit_description='', oid='5c41c84d81c9b68bad4f428a612ad1de0bfdb6d4', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)